#Модули

In [ ]:
#import catboost as cb
import datetime
import gc
import lightgbm as lgb
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import random
import seaborn as sns #Graph library: matplot в фоновом режиме
import sys
import tqdm

In [ ]:
from google.colab import files

from imblearn.pipeline import make_pipeline as imb_make_pipeline
from imblearn.under_sampling import RandomUnderSampler
from imblearn.ensemble import BalancedBaggingClassifier#, EasyEnsemble

from joblib import dump, load

from sklearn.preprocessing import RobustScaler, FunctionTransformer #Imputer,

from sklearn.impute import SimpleImputer
Imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import (roc_auc_score, confusion_matrix, accuracy_score, roc_curve,
                             precision_recall_curve, f1_score, auc)
from sklearn.pipeline import make_pipeline

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

pd.set_option('display.max_columns', None)

#Установки

In [ ]:
TRAIN_TARGET_PATH = "/content/train_target.csv"
TRAIN_DATA_PATH = "/content/train_data/"
TEST_DATA_PATH = "/content/test_data/"

TRAIN_FEATURES_PATH = "/content/train_features_gb/"
TEST_FEATURES_PATH = "/content/test_features_gb/"

In [ ]:
#! rm -r $TRAIN_FEATURES_PATH
! mkdir $TRAIN_FEATURES_PATH
#! rm -r $TEST_FEATURES_PATH
! mkdir $TEST_FEATURES_PATH

! mkdir $TRAIN_DATA_PATH
! mkdir $TEST_DATA_PATH

In [ ]:
!pip install utils

In [ ]:
import utils
from utils import read_parquet_dataset as read_parquet_dataset_from_local
dir(utils)

#Загрузка

In [ ]:
df_train_target = pd.read_csv("train_target.csv")
df_parquet = pd.DataFrame()
for i in range(5):
  df_row = pd.read_parquet(f'{TRAIN_DATA_PATH}train_data_{i}.pq')
  df_parquet = pd.concat([df_parquet, df_row], ignore_index=True)
  del(df_row)

df_parquet = pd.read_parquet('/content/train_data/train_data_0.pq')
#df_parquet
'''
print(df_parquet.nunique())
df_parquet=df_parquet#.head(300000)
df_parquet#.head(10)
'''

In [ ]:
train_target = pd.read_csv(TRAIN_TARGET_PATH)

In [ ]:
df_all=df_train_target.merge(df_parquet)

#Проверки

In [ ]:
df_all.info()
df_all.isna().sum().sum()
print(df_all.size, df_all.shape)

del(df_train_target)
del(df_parquet)
gc.collect()

In [ ]:
neg = df_all[df_all["flag"] == 1].shape[0] #Дефолт
pos = df_all[df_all["flag"] == 0].shape[0]
print(pos)
print(neg)

plt.figure(figsize=(8, 6))

index = [0,1]
values = [pos,neg]
plt.bar(index,values)

plt.xticks((0, 1), ["Ок", "Дефолт"])
plt.xlabel("")
plt.ylabel("Число заемщиков")

In [ ]:
anext = df_all.loc[:, df_all.columns.isin(["flag", 'enc_loans_credit_status'])]#.values
print(anext)
#anext.plot(x="flag", y=["enc_loans_credit_status"])
#plt.plot(anext["flag"], anext["enc_loans_credit_status"])
plt.plot(anext["enc_loans_credit_status"], anext["flag"])
plt.show()

In [ ]:
col_except = ["id"] #"rn"
df_all = df_all.loc[:, ~df_all.columns.isin(col_except)]
df_all

In [ ]:
col_importances = ["flag", 'pre_loans_total_overdue', 'pre_loans_max_overdue_sum',
                   'pre_loans_credit_cost_rate', 'pre_loans_credit_limit',
                   'pre_loans_next_pay_summ', 'pre_loans_outstanding',
                   'pre_loans90', 'pre_loans6090', 'pre_loans3060', 'pre_loans5',
                   'pre_loans530', 'pre_util', 'pre_over2limit', 'pre_maxover2limit',
                   'enc_loans_account_holder_type', 'enc_loans_credit_status', 'enc_loans_account_cur',
                   'enc_loans_credit_type']

df_in = df_all#.loc[:, df_all.columns.isin(col_importances)]
df_in

In [ ]:
probe_col = 'enc_loans_credit_status'
df=df_in[['flag', probe_col]]

df_0 = df[df['flag'] == 0].groupby(probe_col).count()
df_0.reset_index(inplace=True)
df_0.rename(columns = {'flag': 'flag_0'}, inplace = True)

df_1 = df[df['flag'] == 1].groupby(probe_col).count()
df_1.reset_index(inplace=True)
df_1.rename(columns = {'flag': 'flag_1'}, inplace = True)

df_01=df_0.merge(df_1, how='outer').fillna(0)
df_01['flag_0']=df_01['flag_0'].astype(int)

columns=df_01.columns.values.tolist()
df_01[columns].plot(x=probe_col, kind='bar')
plt.show()

In [ ]:
col_except = ["flag"]

X = df.loc[:, ~df.columns.isin(col_except)].values #df.columns != "flag"
y = df.loc[:, df.columns.isin(col_except)].values.flatten() #df.columns == "flag"

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, shuffle=True,
                                                    random_state=0, stratify=y)
print("Оригинальные размеры данных:", X_train.shape, X_test.shape)

columns=df.loc[:, ~df.columns.isin(col_except)].columns.values.tolist()
len(columns)

In [ ]:
train_indices_na = np.max(np.isnan(X_train), axis=1)
test_indices_na = np.max(np.isnan(X_test), axis=1)
X_train_dropna, y_train_dropna = X_train[~train_indices_na, :][:, :-6], y_train[~train_indices_na]
X_test_dropna, y_test_dropna = X_test[~test_indices_na, :][:, :-6], y_test[~test_indices_na]
print("После выкидывания NA: ", X_train_dropna.shape, X_test_dropna.shape)

In [ ]:
def importances_indices(rf_clf, df_in):
  importances = rf_clf.feature_importances_
  indices = np.argsort(importances)[::-1]
  skl_imp = pd.Series(importances[indices],
                         df_in.drop(['flag'], axis=1).columns[indices])
  fig, ax = plt.subplots(figsize=(16,14))
  skl_imp.plot.bar(ax=ax)
  ax.set_title("Важность признаков")
  ax.set_ylabel('Важность')
  fig.tight_layout()
  print(df_in.drop(['flag'], axis=1).columns[indices])

importances_indices(rf_clf, df_in)

#Агрегатор

In [ ]:
class CountAggregator(object):

    def __init__(self):
        self.encoded_features = None

    def __extract_count_aggregations(self, data_frame: pd.DataFrame, mode: str) -> pd.DataFrame:
        # one-hot-encoding
        feature_columns = list(data_frame.columns.values)
        feature_columns.remove("id")
        feature_columns.remove("rn")

        dummies = pd.get_dummies(data_frame[feature_columns], columns=feature_columns)
        dummy_features = dummies.columns.values

        ohe_features = pd.concat([data_frame, dummies], axis=1)
        ohe_features = ohe_features.drop(columns=feature_columns)

        # count aggregation
        ohe_features.groupby("id")
        features = ohe_features.groupby("id")[dummy_features].sum().reset_index(drop=False)
        return features

    def __transform_data(self, path_to_dataset: str, num_parts_to_preprocess_at_once: int = 1, num_parts_total: int=50,
                                     mode: str = "fit_transform", save_to_path=None, verbose: bool=False):
        assert mode in ["fit_transform", "transform"], f"Unrecognized mode: {mode}! Please use one of the following modes: \"fit_transform\", \"transform\""
        preprocessed_frames = []
        for step in tqdm.tqdm_notebook(range(0, num_parts_total, num_parts_to_preprocess_at_once),
                                       desc="Transforming sequential credit data"):
            data_frame = read_parquet_dataset_from_local(path_to_dataset, start_from=step,
                                                         num_parts_to_read=num_parts_to_preprocess_at_once,
                                                         verbose=verbose)
            features = self.__extract_count_aggregations(data_frame, mode=mode)
            if save_to_path:
                features.to_parquet(os.path.join(save_to_path, f"processed_chunk_{step}.pq"))
            preprocessed_frames.append(features)

        features = pd.concat(preprocessed_frames)
        features.fillna(np.uint8(0), inplace=True)
        dummy_features = list(features.columns.values)
        dummy_features.remove("id")
        if mode == "fit_transform":
            self.encoded_features = dummy_features
        else:
            assert not self.encoded_features is None, "Transformer not fitted"
            for col in self.encoded_features:
                if not col in dummy_features:
                    features[col] = np.uint8(0)
        return features[["id"]+self.encoded_features]

    def fit_transform(self, path_to_dataset: str, num_parts_to_preprocess_at_once: int = 1, num_parts_total: int = 50,
                      save_to_path=None, verbose: bool=False):
        return self.__transform_data(path_to_dataset=path_to_dataset,
                                     num_parts_to_preprocess_at_once=num_parts_to_preprocess_at_once,
                                     num_parts_total=num_parts_total, mode="fit_transform",
                                     save_to_path=save_to_path, verbose=verbose)
    def transform(self, path_to_dataset: str, num_parts_to_preprocess_at_once: int = 1, num_parts_total: int=50,
                  save_to_path=None, verbose: bool=False):
        return self.__transform_data(path_to_dataset=path_to_dataset,
                                     num_parts_to_preprocess_at_once=num_parts_to_preprocess_at_once,
                                     num_parts_total=num_parts_total, mode="transform",
                                     save_to_path=save_to_path, verbose=verbose)

In [ ]:
#из утилс - там ее больше нет

def read_parquet_dataset_from_local(path_to_dataset: str, start_from: int = 0,
                                     num_parts_to_read: int = 1, columns=None, verbose=False) -> pd.DataFrame:
    """
    Reads num_parts_to_read parquet partitions and returns the resulting pd.DataFrame
    :param path_to_dataset: directory with parquet partitions
    :param start_from: partition number to start with
    :param num_parts_to_read: amount of partitions to read
    :param columns: columns to read and include
    :return: pd.DataFrame
    """

    res = []
    dataset_paths = sorted([os.path.join(path_to_dataset, filename) for filename in os.listdir(path_to_dataset)
                              if filename.startswith('part')])

    start_from = max(0, start_from)
    if num_parts_to_read < 0:
        chunks = dataset_paths[start_from: ]
    else:
        chunks = dataset_paths[start_from: start_from + num_parts_to_read]
    if verbose:
        print('Reading chunks:\n')
        for chunk in chunks:
            print(chunk)
    for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):
        chunk = pd.read_parquet(chunk_path, columns=columns)
        for col_name, col_type in [('amnt', 'float32'), ('hour_diff', 'int32')]:
            if col_name in chunk.columns:
                chunk[col_name] = chunk[col_name].astype(col_type)

        res.append(chunk)
    return pd.concat(res).reset_index(drop=True)

In [ ]:
%%time
aggregator = CountAggregator()
train_data = aggregator.fit_transform(TRAIN_DATA_PATH, num_parts_to_preprocess_at_once=2,
                                      num_parts_total=2,
                                      save_to_path=TRAIN_FEATURES_PATH, verbose=True)
#num_parts_total=12, num_parts_to_preprocess_at_once=4,

#Модели

##Градиентный бустинг

In [ ]:
models = []
cv = KFold(n_splits=5, random_state=100, shuffle=True)
tree_params = {
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.05,
    "max_depth": 15, #5, 15
    "reg_lambda": 1,
    "num_leaves": 64,
    "n_jobs": -1,#5
    "n_estimators": 80#2000
}

In [ ]:
for i in range(12):
  train_data = pd.read_parquet(TRAIN_DATA_PATH+f'train_data_{i}.pq')
  train_data_target = train_target.merge(train_data, on="id")
  del(train_data)
  gc.collect()
  targets = train_data_target["flag"].values
  oof = np.zeros(len(train_data_target))
  train_preds = np.zeros(len(train_data_target))
  feature_cols = list(train_data_target.columns.values)
  feature_cols.remove("id"), feature_cols.remove("flag")

  for fold_, (train_idx, val_idx) in enumerate(cv.split(train_data_target, targets), 1):
      print(f"Training with fold {fold_} started")
      lgb_model = lgb.LGBMClassifier(**tree_params)
  #    lgb_model = RandomForestClassifier(n_estimators = 10, n_jobs=-1, max_features=10,
  #                                class_weight="balanced", criterion="gini", warm_start = True)
      train, val = train_data_target.iloc[train_idx], train_data_target.iloc[val_idx]

  #    lgb_model.fit(RobustScaler().fit_transform(train[feature_cols], train.flag.values),
  #               train.flag.values)
      lgb_model.fit(train[feature_cols], train.flag.values,
                    eval_set=[(val[feature_cols], val.flag.values)],
                    callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=True)]) #early_stopping_rounds=50, verbose=50

      oof[val_idx] = lgb_model.predict_proba(val[feature_cols])[:, 1]
      train_preds[train_idx] += lgb_model.predict_proba(train[feature_cols])[:, 1] / (cv.n_splits-1)
      models.append(lgb_model)
      print(f"Training with fold {fold_} completed")
  del(train_data_target)
  gc.collect()
  print(i, "-----Train roc-auc: ", roc_auc_score(targets, train_preds))
  print(i, "-----CV roc-auc: ", roc_auc_score(targets, oof))
del(train_target)
gc.collect()

In [ ]:
df_parquet = pd.read_parquet(TEST_DATA_PATH + 'test_data_0.pq')
df_row = pd.read_parquet(TEST_DATA_PATH + 'test_data_1.pq')
test_data = pd.concat([df_parquet, df_row], ignore_index=True)
test_data

In [ ]:
del(df_parquet)
del(df_row)
gc.collect()

In [ ]:
score = np.zeros(len(test_data))

for model in tqdm.tqdm_notebook(models):
    score += model.predict_proba(test_data[feature_cols])[:, 1] / len(models)

submission = pd.DataFrame({
    "id" : test_data["id"].values,
    "score": score
})
submission
#submission.to_csv("submission_lgb.csv", index=None)
#print("Train roc-auc: ", roc_auc_score(targets, train_preds))

In [ ]:
df_result=submission.groupby('id').mean()
df_result.reset_index(inplace=True)
df_result['id'] = df_result['id'].astype(int)
df_result.to_csv('df_result.csv', index=False)
df_result

##Случайный лес

In [ ]:
n_cycle = df_in.shape[0]#100000
K_cycle = df_in.shape[0]//n_cycle
K_cycle_0 = 0 #random.randrange(0, K_cycle)
K_cycle = 1
K_cycle
n_cycle

In [ ]:
def roc_auc_RandomForestClassifier(rf_clf, X_train_dropna, y_train_dropna, X_test_dropna, y_test_dropna):
  #RandomForestClassifier
  probas = rf_clf.fit(X_train_dropna, y_train_dropna).predict_proba(X_test_dropna)
  fpr, tpr, thresholds = roc_curve(y_test_dropna, probas[:, 1])
  roc_auc = auc(fpr, tpr)

  plt.plot(fpr, tpr, label='%s ROC (area = %0.2f)' % ('RandomForest', roc_auc))
  plt.plot([0, 1], [0, 1], 'k--')
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.0])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.legend(loc=0, fontsize='small')
  plt.show()
  return roc_auc
#roc_auc_RandomForestClassifier(rf_clf, X_train_dropna, y_train_dropna, X_test_dropna, y_test_dropna)

In [ ]:
#rf_clf = load('filename_01.joblib')
print(0, datetime.datetime.now())
col_except = ["flag"]
pip_baseline = make_pipeline(RobustScaler(), rf_clf)

for i in range(K_cycle_0, K_cycle_0+K_cycle):
  rf_clf = RandomForestClassifier(n_estimators = 50, n_jobs=-1, max_features=10,
                                class_weight="balanced", criterion="gini")
#criterion="entropy"gini , warm_start = True
#  if i > K_cycle_0:
#    rf_clf = pickle.loads(rf_clf_save)
  X = df_in.iloc[i*n_cycle:(i+1)*n_cycle, ~df_in.columns.isin(col_except)].values #df_in.columns != "flag"
  y = df_in.iloc[i*n_cycle:(i+1)*n_cycle, df_in.columns.isin(col_except)].values.flatten() #df_in.columns == "flag"

  X_train_dropna, X_test_dropna, y_train_dropna, y_test_dropna = train_test_split(X, y,
    test_size=0.3, shuffle=True, random_state=0, stratify=y)

  rf_clf.fit(RobustScaler().fit_transform(Imputer.fit_transform(X_train_dropna)), y_train_dropna)
#  scores = cross_val_score(pip_baseline, X_train_dropna, y_train_dropna, scoring="roc_auc", n_jobs=-1, cv=5)
#  print(i, "Среднее значение AUC базовой модели {}".format(scores.mean()))

  #rf_clf_save = pickle.dumps(rf_clf)
  filename_model = f'filename_{"0"+str(i+1) if i<9 else i+1}.joblib'
  dump(rf_clf, filename_model)
  files.download(filename_model) #('/content/filename.joblib')
  roc_auc = roc_auc_RandomForestClassifier(rf_clf, X_train_dropna, y_train_dropna, X_test_dropna, y_test_dropna)
  dt_now = datetime.datetime.now()
  print(i+1, dt_now, 'roc_auc =', roc_auc)


In [ ]:
rf_clf = load('filename_05.joblib')

test = pd.read_csv("test_target.csv")
#result = pd.DataFrame(test.id)
test_0 = test
test

In [ ]:
df_parquet = pd.read_parquet('/content/test_data_0.pq')
df_row = pd.read_parquet('/content/test_data_1.pq')
df = pd.concat([df_parquet, df_row], ignore_index=True)
df

In [ ]:
del(df_parquet)
del(df_row)
gc.collect()

In [ ]:
col_except = ["id", "rn"]
scores_all = []
roc_auc_all = []
n_cycle = 100000
K_cycle = df.shape[0]//n_cycle
print(0, datetime.datetime.now())

#for i in range(K_cycle):
X_result = df.iloc[:, ~df.columns.isin(col_except)].values

# Формируем предсказания для тестовых данных
#predictions = rf_clf.predict_proba(test)[:, 1]
y_result = rf_clf.predict_proba(X_result) #predict_proba - вероятность, predict - прогноз
#  scores = cross_val_score(pip_baseline, X_result, y_result, scoring="roc_auc", cv=10)
#  roc_auc_all = roc_auc_all.extend(scores)
#  scores_all = scores_all.extend(y_result)
#  print(i, "Среднее значение AUC целевой модели {}".format(scores.mean()))
dt_now = datetime.datetime.now()
print(dt_now)
#print(i+1, dt_now)
print(y_result)

In [ ]:
scores_all = y_result[:, 1]
#?В ексель надо выводить: GINI = 2 ROC AUC — 1
gini_all = [2*i-1 for i in scores_all]

scores_df = pd.DataFrame(scores_all, columns =['score'])
scores_df

s=pd.DataFrame(np.column_stack([df['id'].values, scores_all]), columns =['id', 'score'])
s

In [ ]:
df_result=s.groupby('id').mean()
df_result.reset_index(inplace= True )
df_result['id'] = df_result['id'].astype(int)
df_result.to_csv('df_result.csv', index=False)
df_result

##Прочее

In [ ]:
resampled_rf = BalancedBaggingClassifier(base_estimator=rf_clf,
                                         n_estimators=10,
                                         random_state=0)
'''
pip_resampled = make_pipeline(Imputer(strategy="mean"),
                              RobustScaler(),
                              resampled_rf)
scores = cross_val_score(pip_resampled,
                         X_train, y_train,
                         scoring="roc_auc", cv=10)

print("AUC модели EasyEnsemble: ", scores.mean())
'''

In [ ]:
resampled_rf.fit(X_train_dropna, y_train_dropna)
print(y_test_dropna[-3], y_test_dropna[-2])
print(resampled_rf.predict([X_test_dropna[-3]]), resampled_rf.predict([X_test_dropna[-2]]))

print(y_test_dropna[-3], y_test_dropna[-2])
print(resampled_rf.predict([X_test_dropna[-3]]), resampled_rf.predict([X_test_dropna[-2]]))

In [ ]:
g = df_parquet.columns.to_series().groupby(df_parquet.dtypes).groups
g

df_parquet.groupby('enc_loans_account_cur').mean()
print(df_parquet.isnull().sum())

In [ ]:
df_credit = df_parquet.head(10000)
sns.countplot(df_credit['pre_loans_credit_cost_rate'])

#Результаты